In [2]:
import matplotlib.pyplot as plt
from matplotlib import rc
import pandas as pd
import numpy as np
import seaborn as sns
from sentence_transformers import SentenceTransformer, util
import torch

df = pd.read_csv('/content/drive/MyDrive/fullstack_ai/data/job_cleardata2.csv', encoding='cp949')
print(df)

     대분류(24) 중분류(81) 소분류(279)  세분류(1,114)               직업명
0       사업관리    사업관리   프로젝트관리  공적개발원조사업관리       정부·공공행정 전문가
1       사업관리    사업관리   프로젝트관리  공적개발원조사업관리       국가·지방행정 사무원
2       사업관리    사업관리   프로젝트관리  공적개발원조사업관리          공공행정 사무원
3       사업관리    사업관리   프로젝트관리      프로젝트관리        기획·마케팅 사무원
4       사업관리    사업관리   프로젝트관리      산학협력관리  총무 사무원 및 대학 행정조교
...      ...     ...      ...         ...               ...
1754    농림어업      수산   수산자원관리         염생산       농림어업 단순 종사원
1755    농림어업      수산     어촌개발    어촌체험상품개발        기획·마케팅 사무원
1756    농림어업      수산     어촌개발    어촌체험상품개발          여행상품 개발자
1757    농림어업      수산     어촌개발    어촌체험시설운영  총무 사무원 및 대학 행정조교
1758    농림어업      수산     어촌개발      어업환경개선    환경공학 기술자 및 연구원

[1759 rows x 5 columns]


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
print("Sentence-BERT 모델 로딩 중... 잠시만 기다려 주세요.")
# 사용 가능한 유효한 한국어 모델 이름을 사용합니다.
embedding_model = SentenceTransformer('jhgan/ko-sroberta-multitask')
print("모델 로딩 완료!")

# NCS 직업군 설명 임베딩 (한 번만 수행)
job_descriptions = df['세분류(1,114)'].tolist()
job_embeddings = embedding_model.encode(job_descriptions, convert_to_tensor=True)

Sentence-BERT 모델 로딩 중... 잠시만 기다려 주세요.


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

모델 로딩 완료!


In [4]:
def recommend_jobs():
    print("\n=== NCS 직업군 추천 시스템 ===")
    # 이름, 나이, 성별은 추천 로직에 직접 사용되지는 않지만 사용자에게 친근하게 다가가기 위해 입력받습니다.
    name = input("이름을 입력하세요: ").strip()
    age = input("나이를 입력하세요: ").strip()
    gender = input("성별을 입력하세요 (남/여): ").strip()
    skills = input("Q1. 당신이 목표로 하고있는 직업군 분야를 작성해주세요. : ").strip()
    skills2 = input("Q2. 해당 목표를 위해 실행했던 활동(공모전 참여)이나 실적(자격증 보유 여부 등)이 있나요?  ").strip()
    skills3 = input("Q3. 직군을 목표로 하는 이유가 있다면 말씀해주세요. ").strip()

    if not skills: # 기술/경험 입력은 필수
        print("당신이 목표로 하고있는 직업군 분야를 추천이 가능합니다.")
        return

    if not skills2: # 기술/경험 입력은 필수
        print("해당 목표를 위해 실행했던 활동(공모전 참여)이나 실적(자격증 보유 여부 등) 정보를 입력해야 추천이 가능합니다.")
        return

    if not skills3: # 기술/경험 입력은 필수
        print("직군을 목표로 하는 이유를 입력해야 추천이 가능합니다.")
        return

    # 입력 정보를 하나의 텍스트로 결합 (임베딩 모델 입력으로 사용)
    # 이름, 나이, 성별 정보를 포함하면 모델이 불필요한 정보를 학습할 수 있으므로, 여기서는 기술 및 경험에 집중합니다.
    user_profile_text = f"{skills} {skills2} {skills3}"

    print(f"\n입력된 기술 및 경험: {user_profile_text}")
    print("추천 직업군을 분석 중입니다...")

    # 사용자 프로필 임베딩
    user_embedding = embedding_model.encode(user_profile_text, convert_to_tensor=True)

    # NCS 직업군 임베딩과의 코사인 유사도 계산
    cosine_scores = util.cos_sim(user_embedding, job_embeddings)[0]

    # 결과 DataFrame 생성 및 정렬
    recommendation_df = pd.DataFrame({
        'job_name': df['직업명'],
        'description': df['세분류(1,114)'],
        'similarity_score': cosine_scores.cpu().numpy()
    })
    recommendation_df = recommendation_df.sort_values(by='similarity_score', ascending=False).reset_index(drop=True)

    # 추천 결과 출력
    print("\n=== 당신을 위한 추천 NCS 직업군 Top 5 ===")
    for idx, row in recommendation_df.head(5).iterrows():
        print(f"\n{idx+1}. 직업명: {row['job_name']}")
        print(f"   유사도: {row['similarity_score']:.4f}")
        print(f"   직업분류: {row['description'][:150]}") # 내용이 길 경우 일부만 출력

    # 사용자에게 다음 단계를 제안
    print("\n추천 결과가 만족스러우신가요? 추가로 궁금한 점이 있거나 다른 시도를 해보고 싶으시다면 알려주세요.")

In [6]:
if __name__ == "__main__":
    recommend_jobs()


=== NCS 직업군 추천 시스템 ===
이름을 입력하세요: 손한이
나이를 입력하세요: 25
성별을 입력하세요 (남/여): 남
Q1. 당신이 목표로 하고있는 직업군 분야를 작성해주세요. : 빅데이터
Q2. 해당 목표를 위해 실행했던 활동(공모전 참여)이나 실적(자격증 보유 여부 등)이 있나요?  없음
Q3. 직군을 목표로 하는 이유가 있다면 말씀해주세요. AI 산업에 관심이 많습니다

입력된 기술 및 경험: 빅데이터 없음 AI 산업에 관심이 많습니다
추천 직업군을 분석 중입니다...

=== 당신을 위한 추천 NCS 직업군 Top 5 ===

1. 직업명: 컴퓨터시스템 전문가
   유사도: 0.4497
   직업분류: 인공지능서비스기획

2. 직업명: 응용 소프트웨어 개발자
   유사도: 0.4392
   직업분류: 인공지능학습데이터 구축

3. 직업명: 데이터 전문가
   유사도: 0.4392
   직업분류: 인공지능학습데이터 구축

4. 직업명: 시스템 소프트웨어 개발자
   유사도: 0.4333
   직업분류: 인공지능플랫폼구축

5. 직업명: 컴퓨터시스템 전문가
   유사도: 0.4333
   직업분류: 인공지능플랫폼구축

추천 결과가 만족스러우신가요? 추가로 궁금한 점이 있거나 다른 시도를 해보고 싶으시다면 알려주세요.
